In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [14]:
df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")
print(df_train.shape, df_test.shape)

(1460, 81) (1459, 80)


In [15]:
df_train.drop("Id", axis=1, inplace=True)
test_ID = df_test['Id']
df_test.drop("Id", axis=1, inplace=True)
sale_price = df_train.pop("SalePrice")

In [16]:
all_data = pd.concat((df_train,df_test))
all_data.shape

(2919, 79)

In [17]:
count = all_data.shape[0]
s = all_data.isnull().sum() / count * 100 
s.sort_values(ascending=False).drop(s[s == 0].index)

PoolQC          99.657417
MiscFeature     96.402878
Alley           93.216855
Fence           80.438506
FireplaceQu     48.646797
LotFrontage     16.649538
GarageCond       5.447071
GarageQual       5.447071
GarageYrBlt      5.447071
GarageFinish     5.447071
GarageType       5.378554
BsmtCond         2.809181
BsmtExposure     2.809181
BsmtQual         2.774923
BsmtFinType2     2.740665
BsmtFinType1     2.706406
MasVnrType       0.822199
MasVnrArea       0.787941
MSZoning         0.137033
BsmtHalfBath     0.068517
Utilities        0.068517
Functional       0.068517
BsmtFullBath     0.068517
BsmtFinSF2       0.034258
BsmtFinSF1       0.034258
Exterior2nd      0.034258
BsmtUnfSF        0.034258
TotalBsmtSF      0.034258
Exterior1st      0.034258
SaleType         0.034258
Electrical       0.034258
KitchenQual      0.034258
GarageArea       0.034258
GarageCars       0.034258
dtype: float64

In [18]:
all_data['PoolQC'] = all_data['PoolQC'].fillna('NA')
all_data.loc[all_data.PoolQC!='NA', 'PoolQC']

197     Ex
810     Fa
1170    Gd
1182    Ex
1298    Gd
1386    Fa
1423    Gd
514     Ex
1113    Ex
1250    Gd
Name: PoolQC, dtype: object

In [19]:
all_data['MiscFeature'] = all_data['MiscFeature'].fillna('NA')
all_data['Alley'] = all_data['Alley'].fillna('NA')
all_data['Fence'] = all_data['Fence'].fillna('NA')
all_data['FireplaceQu'] = all_data['FireplaceQu'].fillna('NA')
all_data['LotFrontage'] = all_data['LotFrontage'].fillna(0)
for g in  ['GarageCond', 'GarageQual', 'GarageFinish', 'GarageType']:
    all_data[g] = all_data[g].fillna('NA')
for g in ['GarageCars','GarageArea', 'GarageYrBlt']:
    all_data[g] = all_data[g].fillna(0)
for g in ['BsmtCond','BsmtQual','BsmtExposure','BsmtFinType1','BsmtFinType2']:
    all_data[g] = all_data[g].fillna('NA')
for g in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']:
    all_data[g] = all_data[g].fillna(0)
all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna('RH')
all_data['Utilities'] = all_data['Utilities'].fillna('AllPub')
all_data['Functional'] = all_data['Functional'].fillna('Typ')
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode().iloc[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode().iloc[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode().iloc[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode().iloc[0])
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode().iloc[0])
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [20]:
s = all_data.isnull().sum()
s.loc[s>0]

Series([], dtype: int64)

In [21]:
all_data['MSSubClass'] = all_data['MSSubClass'].astype(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [22]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
for c in cols:
    le.fit(all_data[c].values)
    all_data[c] = le.transform(all_data[c].values)

In [23]:
all_data = pd.get_dummies(all_data)


In [24]:
n_train = df_train.shape[0]
new_train = all_data[:n_train]
new_test = all_data[n_train:]

In [25]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [26]:
y_train = sale_price.values
y_train

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [27]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42)
    rmse= np.sqrt(-cross_val_score(model, new_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)


In [28]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [29]:
# score = rmsle_cv(lasso)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [30]:
# score = rmsle_cv(ENet)
# print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [31]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 27429.9766 (7749.3704)



In [32]:
b = [1,2,3,4,5]
a = [list() for x in b]
a

[[], [], [], [], []]

In [33]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [34]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),meta_model = lasso)
score = rmsle_cv(stacked_averaged_models) # there is a bug which the kfold is used twice
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: 27294.2506 (8155.8967)


In [35]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [36]:
stacked_averaged_models.fit(new_train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(new_train.values)
stacked_pred = stacked_averaged_models.predict(new_test.values)
print(rmsle(y_train, stacked_train_pred))

15673.4642338


In [37]:
import xgboost as xgb
import lightgbm as lgb
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)


In [38]:
model_xgb.fit(new_train, y_train)
xgb_train_pred = model_xgb.predict(new_train)
xgb_pred = model_xgb.predict(new_test)
print(rmsle(y_train, xgb_train_pred))

3587.49717316


In [50]:
model_lgb.fit(new_train, y_train)
lgb_train_pred = model_lgb.predict(new_train)
lgb_pred = model_lgb.predict(new_test)
print(rmsle(y_train, lgb_train_pred))

48745.4963711


In [51]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [52]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = ensemble
sub.to_csv('submission.csv',index=False)

In [53]:
xgb_pred

array([ 126960.7109375,  159945.       ,  194388.671875 , ...,
        161788.375    ,  115411.6484375,  212928.484375 ], dtype=float32)